# Sheet properties

In [12]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [13]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [14]:
%matplotlib inline
# produce vector inline graphics
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
import matplotlib.pyplot as plt

# Start with local

In [15]:
import sys
from IPython.display import HTML
sys.path.append('/home/rcendre/classification')

# Imports

In [16]:
import os
import sys
import itertools
import webbrowser
from pathlib import Path
import matplotlib.pyplot as plt
from numpy import array, logspace
from scipy.stats import randint,uniform
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import davies_bouldin_score
from toolbox.classification.common import Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.models.builtin import Applications
from toolbox.IO import dermatology
from toolbox.transforms.at_most import LDAAtRatio
from toolbox.transforms.common import PredictorTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.transforms.images import DistributionImageTransform, DWTImageTransform, FourierImageTransform, HaralickImageTransform, SpatialImageTransform
from toolbox.views.common import Views, ViewsTools
from toolbox.views.images import ImagesViews
from IPython.display import display

# Parameters

In [17]:
# Advanced parameters
data_type='Full'
validation = 4
settings = Settings.get_default_dermatology()

# Inputs

In [18]:
features_file = f'Transfer_{data_type}.pickle'
prediction_file = f'PredictionTransferReduction_{data_type}.pickle'

In [19]:
inputs = IO.load(features_file)

# Features extraction

In [20]:
extractors = [('VGG16Max', {}), ('InceptionV3Max', {}), ('InceptionResNetV2Max', {}), ('ResNetMax', {}),
              ('VGG16Avg', {}), ('InceptionV3Avg', {}), ('InceptionResNetV2Avg', {}), ('ResNetAvg', {})]

# Classifier

In [21]:
clfs = []

# SVM Linear
clf = ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))
clf_params = {'clf__C': logspace(-2, 3, 6).tolist()}
clfs.append(('SVML', clf, clf_params))

# SVM RBF
clf = ('clf', SVC(kernel='rbf', class_weight='balanced', probability=True))
clf_params = {'clf__C': logspace(-2, 3, 6).tolist(),
             'clf__gamma': logspace(-2, 3, 6).tolist()}
clfs.append(('SVMR', clf, clf_params))

# CART
clf = ('clf', DecisionTreeClassifier(class_weight='balanced'))
clf_params = {'clf__max_depth': randint(4, 10),
              'clf__max_features': randint(1, 9),
              'clf__min_samples_leaf': randint(1, 9),
              'clf__criterion': ['gini', 'entropy']}
clfs.append(('CART', clf, clf_params))

# Extra
clf = ('clf', ExtraTreesClassifier(n_estimators=100 ,class_weight='balanced'))
clf_params = {'clf__max_depth': randint(4, 10),
              'clf__max_features': randint(1, 9),
              'clf__min_samples_leaf': randint(1, 9),
              'clf__criterion': ['gini', 'entropy']}
clfs.append(('EXTRA', clf, clf_params))

# GB
clf = ('clf', GradientBoostingClassifier(n_estimators=100))
clf_params = {'clf__max_depth': randint(4, 10),
              'clf__max_features': randint(1, 9),
              'clf__min_samples_leaf': randint(1, 9)}
clfs.append(('GB', clf, clf_params))

# MLP
clf = ('clf', MLPClassifier())
clf_params = {'clf__hidden_layer_sizes': [(randint.rvs(10,20,1),randint.rvs(10,20,1),),(randint.rvs(10,20,1),)],
                'clf__activation': ['tanh', 'relu'],
                'clf__solver': ['sgd', 'adam'],
                'clf__alpha': uniform(0.0001, 0.9),
                'clf__learning_rate': ['constant','adaptive']}
clfs.append(('MLP', clf, clf_params))

# Models

In [22]:
scalers = [('MMS',('scale', MinMaxScaler())), ('RS',('scale', RobustScaler())), ('SS',('scale', StandardScaler()))]

reductions = [('PCA95',('reduction', PCA(n_components=0.95))), ('PCA975',('reduction', PCA(n_components=0.975))), ('PCA99',('reduction', PCA(n_components=0.99))),
                ('LDA95',('reduction', LDAAtRatio(n_components=0.95))), ('LDA975',('reduction', LDAAtRatio(n_components=0.975))), ('LDA99',('reduction', LDAAtRatio(n_components=0.99)))]

processes = []
for scaler_name,scaler in scalers:

    for reduction_name,reduction in reductions:

        for clf_name, clf, clf_params in clfs:
        
            process = Pipeline([scaler, reduction, clf])
            processes.append((f'{scaler_name}_{reduction_name}_{clf_name}', process, clf_params))
            

# Evaluation

In [ ]:
for extractor in extractors: 
    # Decompose model
    extractor_name, extractor_process = extractor
    
    for process in processes:   
        # Decompose model       
        process_name, process_process, process_params = process
        
        # Start evaluation
        name = f'{extractor_name}{process_name}Label'        
        print(f'{name} performed...', end='\r')
        Tools.evaluate(inputs, {'datum': extractor_name, 'label_encode': 'LabelEncode'}, process_process, name, distribution=process_params)
        print(f'{name} achieved !', end='\r')  
        
        name = f'{extractor_name}{process_name}Malignant'
        print(f'{name} performed...', end='\r')
        Tools.evaluate(inputs, {'datum': extractor_name, 'label_encode': 'MalignantEncode'}, process_process, name, distribution=process_params)
        print(f'{name} achieved !', end='\r')
        
        
        name = f'{extractor_name}{process_name}Pathological'
        print(f'{name} performed...', end='\r')
        Tools.evaluate(inputs, {'datum': extractor_name, 'label_encode': 'PathologicalEncode'}, process_process, name, distribution=process_params)
        print(f'{name} achieved !', end='\r')
        
        # Save
        IO.save(inputs, prediction_file)

/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


/home/rcendre/anaconda3/envs/PythonGPU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


# Scores and ROC

In [ ]:
# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
pathological_encoder = OrderedEncoder().fit(['Normal', 'Pathological'])
malignant_encoder = OrderedEncoder().fit(['Rest', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))
for extractor in extractors: 
    extractor_name, extractor_process = extractor
    
    for model in models:  
        # Decompose model
        model_name, model_process, model_params = model        
        
        # Label
        name = f'{extractor_name}{model_name}Label' 
        display(HTML(ViewsTools.dataframe_renderer([Views.report(ViewsTools.data_as(inputs, name), {'label_encode': 'LabelEncode', 'eval': name}, label_encoder),
                                                                        title=[f'Test - {name}'])))
        # Malignant
        name = f'{extractor_name}{model_name}Malignant'  
        display(HTML(ViewsTools.dataframe_renderer([Views.report(ViewsTools.data_as(inputs, name), {'label_encode': 'MalignantEncode', 'eval': name}, malignant_encoder),
                                    title=[f'Test - {name}'])))  
        
        # Pathologic
        name = f'{extractor_name}{model_name}Pathologic'  
        display(HTML(ViewsTools.dataframe_renderer([Views.report(ViewsTools.data_as(inputs, name), {'label_encode': 'PathologicalEncode', 'eval': name}, pathological_encoder),
                                    title=[f'Test - {name}'])))  
#         display(Views.details(ViewsTools.data_as(inputs, name), {'eval': name}))
#         display(HTML(ViewsTools.dataframe_renderer([Views.report(ViewsTools.data_as(inputs, name), {'label_encode': 'MalignantEncode', 'eval': name}, malignant_encoder),
#                                     Views.report(ViewsTools.data_as(inputs, name, as_train=True), {'label_encode': 'MalignantEncode', 'eval': name}, malignant_encoder)],
#                                     title=[f'Test - {name}', f'Train - {name}'])))   
#         Views.receiver_operator_curves(ViewsTools.data_as(inputs, name), malignant_encoder, {'label_encode': 'MalignantEncode', 'eval': name}, settings);